In [119]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import pandas as pd
import re
import requests
import seaborn as sns
import time
import descartes
import geopandas as gpd

from arcgis.geocoding import geocode
from arcgis.gis import GIS
from dotenv import load_dotenv
from functools import reduce
from shapely.geometry import Point, Polygon
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeRegressor
from textwrap import wrap
from xgboost import XGBClassifier, XGBRegressor, plot_importance
from yellowbrick.classifier import ROCAUC, ConfusionMatrix

plt.style.use("ggplot")
sns.set_context("talk")

In [2]:
load_dotenv("")
censuskey = os.getenv("censuskey")

In [207]:
var_description = pd.DataFrame(pd.DataFrame(
    requests.get("https://api.census.gov/data/2016/acs/acs5/profile/variables.json")
                                            .json()["variables"])
                               .T.iloc[:, 3]
                               .str.title()
                               .str.split("!!", expand=True)
                              ).sort_index()

In [208]:
var_description.columns=["unit", 
                         "subject",
                         "denominator",
                         "criteria_1",
                         "criteria_2",
                         "criteria_3",
                         "criteria_4"
                        ]

In [213]:
var_description

,unit,subject,denominator,criteria_1,criteria_2,criteria_3,criteria_4
AIANHH,American Indian Area/Alaska Native Area/Hawaii...,None,None,None,None,None,None
AIHHTLI,American Indian Area (Off-Reservation Trust La...,None,None,None,None,None,None
AITSCE,American Indian Tribal Subdivision (Census),None,None,None,None,None,None
ANRC,Alaska Native Regional Corporation,None,None,None,None,None,None
CBSA,Metropolitan Statistical Area/Micropolitan Sta...,None,None,None,None,None,None
CD115,Congressional District (115Th),None,None,None,None,None,None
CNECTA,Combined New England City And Town Area,None,None,None,None,None,None
CONCIT,Consolidated City,None,None,None,None,None,None
COUNTY,County (Fips),None,None,None,None,None,None
COUSUB,County Subdivision (Fips),None,None,None,None,None,None


In [210]:
var_description.to_csv("../data/var_description.csv")

In [211]:
variables_selected = var_description.loc[((~var_description.index.str.contains(r'DP')|
                                           (var_description.index.str.contains(r'DP')&
                                            var_description.index.str.contains(r'[0-9]PE')))&
                                          ~var_description.index.str.contains(r'PR_'))
                                        ].sort_index()

In [109]:
len(variables_selected)

554

In [193]:
varlist_dict = {}
var_dict = {}
for year in range(2010, 2018):
    varlist_dict[year] = []
    k = 0
    for var in variables_selected.index:
        success = False
        l = 0
        while not success:
            l += 1
            resp = requests.get(
                "https://api.census.gov/data/{}/acs/acs5/profile?get={}&for=county:*&in=state:*&key={}"
                .format(year, var, censuskey))
            time.sleep(1)
            if resp.status_code ==200:
                varlist_dict[year].append(pd.DataFrame(resp.json()[1:], columns=resp.json()[0]))
                success = True
            elif l > 10:
                success = True
        k += 1
        
        if k%int(len(variables_selected)/10)==0:
            print("---progress: {:0.2f}% completed for {}---".format(
                k*10/int(len(variables_selected)/10), 
                year
            ))
    var_dict[year] = reduce(lambda left, right: pd.merge(left, 
                                                         right, 
                                                         how='outer'
                                                        ), varlist_dict[year])
    print(year, var_dict[year].shape)

---progress: 10.00% completed for 2010---
---progress: 20.00% completed for 2010---
---progress: 30.00% completed for 2010---
---progress: 40.00% completed for 2010---
---progress: 50.00% completed for 2010---
---progress: 60.00% completed for 2010---
---progress: 70.00% completed for 2010---
---progress: 80.00% completed for 2010---
---progress: 90.00% completed for 2010---
---progress: 100.00% completed for 2010---
2010 (3221, 544)
---progress: 10.00% completed for 2011---
---progress: 20.00% completed for 2011---
---progress: 30.00% completed for 2011---
---progress: 40.00% completed for 2011---
---progress: 50.00% completed for 2011---
---progress: 60.00% completed for 2011---
---progress: 70.00% completed for 2011---
---progress: 80.00% completed for 2011---
---progress: 90.00% completed for 2011---
---progress: 100.00% completed for 2011---
2011 (3221, 545)
---progress: 10.00% completed for 2012---
---progress: 20.00% completed for 2012---
---progress: 30.00% completed for 2012--

In [459]:
missing_varlist_dict = {}
missing_var_dict = {}
for year in range(2010, 2018):
    missing_col = []
    for col in var_dict[2016].columns:
        if col not in var_dict[2010].columns:
            missing_col.append(col)
    missing_varlist_dict[year] = []
    k = 0
    for var in missing_col:
        success = False
        l = 0
        while not success:
            l += 1
            resp = requests.get(
                "https://api.census.gov/data/{}/acs/acs5/profile?get={}&for=county:*&in=state:*&key={}"
                .format(year, var, censuskey))
#             time.sleep(1)
            if resp.status_code ==200:
                missing_varlist_dict[year].append(pd.DataFrame(resp.json()[1:], columns=resp.json()[0]))
                success = True
            elif l > 10:
                success = True
                print("no data for: ", year, var)
        k += 1

no data for:  2010 CD115
no data for:  2010 DP02_0150PE
no data for:  2010 DP02_0151PE
no data for:  2010 DP02_0152PE
no data for:  2010 DP04_0142PE
no data for:  2010 DP04_0143PE
no data for:  2010 DP05_0082PE
no data for:  2010 DP05_0083PE
no data for:  2010 DP05_0084PE
no data for:  2010 ZCTA5
no data for:  2011 CD115
no data for:  2011 DP02_0150PE
no data for:  2011 DP02_0151PE
no data for:  2011 DP02_0152PE
no data for:  2011 DP04_0142PE
no data for:  2011 DP04_0143PE
no data for:  2011 DP05_0082PE
no data for:  2011 DP05_0083PE
no data for:  2011 DP05_0084PE
no data for:  2012 CD115
no data for:  2012 DP02_0150PE
no data for:  2012 DP02_0151PE
no data for:  2012 DP02_0152PE
no data for:  2012 DP04_0142PE
no data for:  2012 DP04_0143PE
no data for:  2012 DP05_0082PE
no data for:  2012 DP05_0083PE
no data for:  2012 DP05_0084PE
no data for:  2013 CD115
no data for:  2013 DP04_0142PE
no data for:  2013 DP04_0143PE
no data for:  2013 DP05_0082PE
no data for:  2013 DP05_0083PE
no data

In [467]:
for year in range(2010, 2018):
    var_dict[year] = var_dict[year].apply(pd.to_numeric, errors='coerce')
    var_dict[year] = var_dict[year].loc[var_dict[year].state!=72]
    var_dict[year]["FIPS"] = var_dict[year].state * 1000 + var_dict[year].county
    var_dict[year].to_csv("../data/census_{}.csv".format(year))

In [265]:
pd.options.display.float_format = '{:,.2f}'.format

In [466]:
var_dict[2016].head()

,AIANHH,state,county,AIHHTLI,AITSCE,ANRC,CBSA,CD115,CNECTA,CONCIT,COUNTY,COUSUB,CSA,DIVISION,DP02_0001PE,DP02_0002PE,DP02_0003PE,DP02_0004PE,DP02_0005PE,DP02_0006PE,DP02_0007PE,DP02_0008PE,DP02_0009PE,DP02_0010PE,DP02_0011PE,DP02_0012PE,DP02_0013PE,DP02_0014PE,DP02_0015PE,DP02_0016PE,DP02_0017PE,DP02_0018PE,DP02_0019PE,DP02_0020PE,DP02_0021PE,DP02_0022PE,DP02_0023PE,DP02_0024PE,DP02_0025PE,DP02_0026PE,DP02_0027PE,DP02_0028PE,DP02_0029PE,DP02_0030PE,DP02_0031PE,DP02_0032PE,DP02_0033PE,DP02_0034PE,DP02_0035PE,DP02_0036PE,DP02_0037PE,DP02_0038PE,DP02_0039PE,DP02_0040PE,DP02_0041PE,DP02_0042PE,DP02_0043PE,DP02_0044PE,DP02_0045PE,DP02_0046PE,DP02_0047PE,DP02_0048PE,DP02_0049PE,DP02_0050PE,DP02_0051PE,DP02_0052PE,DP02_0053PE,DP02_0054PE,DP02_0055PE,DP02_0056PE,DP02_0057PE,DP02_0058PE,DP02_0059PE,DP02_0060PE,DP02_0061PE,DP02_0062PE,DP02_0063PE,DP02_0064PE,DP02_0065PE,DP02_0066PE,DP02_0067PE,DP02_0068PE,DP02_0069PE,DP02_0070PE,DP02_0071PE,DP02_0072PE,DP02_0073PE,DP02_0074PE,DP02_0075PE,DP02_0076PE,DP02_0077PE,DP02_0078PE,DP02_0079PE,DP02_0080PE,DP02_0081PE,DP02_0082PE,DP02_0083PE,DP02_0084PE,DP02_0085PE,DP02_0086PE,DP02_0087PE,DP02_0088PE,DP02_0089PE,DP02_0090PE,DP02_0091PE,DP02_0092PE,DP02_0093PE,DP02_0094PE,DP02_0095PE,DP02_0096PE,DP02_0097PE,DP02_0098PE,DP02_0099PE,DP02_0100PE,DP02_0101PE,DP02_0102PE,DP02_0103PE,DP02_0104PE,DP02_0105PE,DP02_0106PE,DP02_0107PE,DP02_0108PE,DP02_0109PE,DP02_0110PE,DP02_0111PE,DP02_0112PE,DP02_0113PE,DP02_0114PE,DP02_0115PE,DP02_0116PE,DP02_0117PE,DP02_0118PE,DP02_0119PE,DP02_0120PE,DP02_0121PE,DP02_0122PE,DP02_0123PE,DP02_0124PE,DP02_0125PE,DP02_0126PE,DP02_0127PE,DP02_0128PE,DP02_0129PE,DP02_0130PE,DP02_0131PE,DP02_0132PE,DP02_0133PE,DP02_0134PE,DP02_0135PE,DP02_0136PE,DP02_0137PE,DP02_0138PE,DP02_0139PE,DP02_0140PE,DP02_0141PE,DP02_0142PE,DP02_0143PE,DP02_0144PE,DP02_0145PE,DP02_0146PE,DP02_0147PE,DP02_0148PE,DP02_0149PE,DP02_0150PE,DP02_0151PE,DP02_0152PE,DP03_0001PE,DP03_0002PE,DP03_0003PE,DP03_0004PE,DP03_0005PE,DP03_0006PE,DP03_0007PE,DP03_0008PE,DP03_0009PE,DP03_0010PE,DP03_0011PE,DP03_0012PE,DP03_0013PE,DP03_0014PE,DP03_0015PE,DP03_0016PE,DP03_0017PE,DP03_0018PE,DP03_0019PE,DP03_0020PE,DP03_0021PE,DP03_0022PE,DP03_0023PE,DP03_0024PE,DP03_0025PE,DP03_0026PE,DP03_0027PE,DP03_0028PE,DP03_0029PE,DP03_0030PE,DP03_0031PE,DP03_0032PE,DP03_0033PE,DP03_0034PE,DP03_0035PE,DP03_0036PE,DP03_0037PE,DP03_0038PE,DP03_0039PE,DP03_0040PE,DP03_0041PE,DP03_0042PE,DP03_0043PE,DP03_0044PE,DP03_0045PE,DP03_0046PE,DP03_0047PE,DP03_0048PE,DP03_0049PE,DP03_0050PE,DP03_0051PE,DP03_0052PE,DP03_0053PE,DP03_0054PE,DP03_0055PE,DP03_0056PE,DP03_0057PE,DP03_0058PE,DP03_0059PE,DP03_0060PE,DP03_0061PE,DP03_0062PE,DP03_0063PE,DP03_0064PE,DP03_0065PE,DP03_0066PE,DP03_0067PE,DP03_0068PE,DP03_0069PE,DP03_0070PE,DP03_0071PE,DP03_0072PE,DP03_0073PE,DP03_0074PE,DP03_0075PE,DP03_0076PE,DP03_0077PE,DP03_0078PE,DP03_0079PE,DP03_0080PE,DP03_0081PE,DP03_0082PE,DP03_0083PE,DP03_0084PE,DP03_0085PE,DP03_0086PE,DP03_0087PE,DP03_0088PE,DP03_0089PE,DP03_0090PE,DP03_0091PE,DP03_0092PE,DP03_0093PE,DP03_0094PE,DP03_0095PE,DP03_0096PE,DP03_0097PE,DP03_0098PE,DP03_0099PE,DP03_0100PE,DP03_0101PE,DP03_0102PE,DP03_0103PE,DP03_0104PE,DP03_0105PE,DP03_0106PE,DP03_0107PE,DP03_0108PE,DP03_0109PE,DP03_0110PE,DP03_0111PE,DP03_0112PE,DP03_0113PE,DP03_0114PE,DP03_0115PE,DP03_0116PE,DP03_0117PE,DP03_0118PE,DP03_0119PE,DP03_0120PE,DP03_0121PE,DP03_0122PE,DP03_0123PE,DP03_0124PE,DP03_0125PE,DP03_0126PE,DP03_0127PE,DP03_0128PE,DP03_0129PE,DP03_0130PE,DP03_0131PE,DP03_0132PE,DP03_0133PE,DP03_0134PE,DP03_0135PE,DP03_0136PE,DP03_0137PE,DP04_0001PE,DP04_0002PE,DP04_0003PE,DP04_0004PE,DP04_0005PE,DP04_0006PE,DP04_0007PE,DP04_0008PE,DP04_0009PE,DP04_0010PE,DP04_0011PE,DP04_0012PE,DP04_0013PE,DP04_0014PE,DP04_0015PE,DP04_0016PE,DP04_0017PE,DP04_0018PE,DP04_0019PE,DP04_0020PE,DP04_0021PE,DP04_0022PE,DP04_0023PE,DP04_0024PE,DP04_0025PE,DP04_0026PE,DP04_0027PE,DP04_0028PE,DP04_0029PE,DP04_0030PE,DP04_0031PE,DP04_0032PE,DP04_0033PE,DP04_0034PE,DP04_0035PE,DP04_0036PE,DP04_00

In [266]:
var_dict[2016].loc[:,(var_dict[2016].max()>100)|(var_dict[2016].min()<0)].describe()

,county,CBSA,COUNTY,CSA,DP02_0001PE,DP02_0015PE,DP02_0016PE,DP02_0017PE,DP02_0024PE,DP02_0030PE,DP02_0036PE,DP02_0037PE,DP02_0038PE,DP02_0039PE,DP02_0040PE,DP02_0041PE,DP02_0042PE,DP02_0043PE,DP02_0044PE,DP02_0045PE,DP02_0046PE,DP02_0047PE,DP02_0048PE,DP02_0049PE,DP02_0050PE,DP02_0051PE,DP02_0052PE,DP02_0058PE,DP02_0068PE,DP02_0070PE,DP02_0072PE,DP02_0074PE,DP02_0076PE,DP02_0078PE,DP02_0086PE,DP02_0093PE,DP02_0094PE,DP02_0095PE,DP02_0096PE,DP02_0097PE,DP02_0098PE,DP02_0099PE,DP02_0100PE,DP02_0101PE,DP02_0102PE,DP02_0103PE,DP02_0104PE,DP02_0105PE,DP02_0106PE,DP02_0107PE,DP02_0108PE,DP02_0109PE,DP02_0110PE,DP02_0122PE,DP02_0150PE,DP02_0151PE,DP02_0152PE,DP03_0001PE,DP03_0008PE,DP03_0010PE,DP03_0014PE,DP03_0015PE,DP03_0016PE,DP03_0017PE,DP03_0018PE,DP03_0025PE,DP03_0026PE,DP03_0032PE,DP03_0046PE,DP03_0051PE,DP03_0062PE,DP03_0063PE,DP03_0065PE,DP03_0067PE,DP03_0069PE,DP03_0071PE,DP03_0073PE,DP03_0075PE,DP03_0086PE,DP03_0087PE,DP03_0088PE,DP03_0089PE,DP03_0090PE,DP03_0091PE,DP03_0092PE,DP03_0093PE,DP03_0094PE,DP03_0095PE,DP03_0100PE,DP03_0102PE,DP03_0103PE,DP03_0104PE,DP03_0109PE,DP03_0110PE,DP03_0111PE,DP03_0112PE,DP03_0113PE,DP03_0114PE,DP03_0120PE,DP03_0121PE,DP03_0123PE,DP03_0124PE,DP03_0125PE,DP03_0126PE,DP03_0127PE,DP03_0129PE,DP03_0130PE,DP03_0131PE,DP03_0132PE,DP04_0001PE,DP04_0004PE,DP04_0005PE,DP04_0006PE,DP04_0016PE,DP04_0027PE,DP04_0037PE,DP04_0038PE,DP04_0045PE,DP04_0048PE,DP04_0049PE,DP04_0050PE,DP04_0057PE,DP04_0062PE,DP04_0072PE,DP04_0073PE,DP04_0075PE,DP04_0076PE,DP04_0080PE,DP04_0089PE,DP04_0090PE,DP04_0093PE,DP04_0094PE,DP04_0095PE,DP04_0096PE,DP04_0097PE,DP04_0098PE,DP04_0099PE,DP04_0100PE,DP04_0101PE,DP04_0102PE,DP04_0109PE,DP04_0110PE,DP04_0111PE,DP04_0112PE,DP04_0113PE,DP04_0114PE,DP04_0115PE,DP04_0116PE,DP04_0117PE,DP04_0125PE,DP04_0126PE,DP04_0134PE,DP04_0135PE,DP04_0136PE,DP04_0143PE,DP05_0001PE,DP05_0017PE,DP05_0022PE,DP05_0025PE,DP05_0028PE,DP05_0058PE,DP05_0065PE,DP05_0081PE,DP05_0082PE,METDIV
count,"3,220.00","3,220.00","3,220.00","3,220.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00"
mean,102.92,"58,578.50",102.92,745.80,"37,465.38","-888,888,888.00","-888,888,888.00","98,816.98","39,970.37","41,989.63","-7,071,403.76","-9,194,390.41","-888,888,888.00","-888,888,888.00","-888,888,888.00","-888,888,888.00","-

In [267]:
var_dict[2016].loc[:,var_dict[2016].min()==-666666666.0].describe()

,DP02_0044PE,DP02_0045PE,DP02_0046PE,DP02_0047PE,DP02_0048PE,DP02_0050PE,DP02_0051PE,DP02_0094PE,DP02_0095PE,DP02_0098PE,DP02_0099PE,DP02_0101PE,DP02_0102PE,DP02_0104PE,DP02_0105PE,DP02_0106PE,DP02_0107PE,DP02_0108PE,DP02_0109PE,DP03_0015PE,DP03_0017PE,DP03_0110PE,DP03_0111PE,DP03_0112PE,DP03_0113PE,DP03_0120PE,DP03_0121PE,DP03_0123PE,DP03_0124PE,DP03_0125PE,DP03_0126PE,DP03_0127PE,DP03_0129PE,DP03_0130PE,DP03_0131PE,DP03_0132PE,DP04_0094PE,DP04_0095PE,DP04_0096PE,DP04_0097PE,DP04_0098PE,DP04_0099PE,DP04_0100PE,DP04_0111PE,DP04_0112PE,DP04_0113PE,DP04_0114PE,DP04_0115PE
count,"3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00","3,220.00"
mean,"-3,819,174.82","-3,819,213.12","-3,819,212.64","-3,819,215.61","-3,819,203.72","-10,821,073.59","-10,821,061.09","-2,758,285.00","-2,758,271.47","-15,913,422.86","-15,913,341.37","-2,758,313.92","-2,758,242.54","-2,758,310.23","-2,758,303.93","-2,758,324.15","-2,758,326.93","-2,758,280.48","-2,758,322.85","-414,013.90","-206,968.87","-2,070,334.90","-2,070,359.00","-2,070,366.53","-2,070,352.16","-207,018.71","-1,449,252.35","-414,068.60","-2,277,423.34","-207,004.74","-414,033.73","-17,391,252.31","-207,015.63","-207,016.03","-414,051.52","-207,017.38","-207,035.16","-207,001.46","-207,007.95","-207,024.86","-207,033.20","-207,036.54","-207,036.23","-206,992.90","-207,024.29","-207,029.23","-207,032.51","-207,017.78"
std,"50,322,647.21","50,322,644.30","50,322,644.33","50,322,644.11","50,322,645.01","84,257,062.09","84,257,063.70","42,800,236.41","42,800,237.28","101,779,240.80","101,779,253.55","42,800,234.54","42,800,239.14","42,800,234.78","42,800,235.19","42,800,233.88","42,800,233.70","42,800,236.70","42,800,233.97","16,612,246.86","11,748,457.51","37,099,911.25","37,099,909.91","37,099,909.49","37,099,910.29","11,748,456.63","31,054,512.37","16,612,245.50","38,904,650.98","11,748,456.88","16,612,246.37","106,279,137.47","11,748,456.69","11,748,456.68","16,612,245.93","11,748,456.66","11,748,456.34","11,748,456.94","11,748,456.82","11,748,456.52","11,748,456.38","11,748,456.32","11,748,456.32","11,748,457.09","11,748,456.53","11,748,456.45","11,748,456.39","11,748,456.65"
min,"-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00","-666,666,666.00"
25%,37.90,4.70,5.30,3.10,11.90,54.60,64.80,30.10,45.40,0.00,87.50,5.70,81.00,6.40,9.33,0.00,0.00,24.50,0.20,58.50,65.50,47.70,24.00,17.20,29.80,13.50,13.30,5.20,2.00,26.98,35.50,36.70,15.60,15.10,17.50,13.90,1.70,27.00,27.10,9.30,2.70,0.70,0.60,40.60,13.30,8.50,5.50,16.80
50%,47.65,8.50,9.00,6.30,18.20,60.15,73.70,42.10,57.90,4.00,95.20,11.50,88.40,14.20,21.00,1.40,0.00,46.40,2.30,65.70,71.20,59.55,33.70,25.10,40.30,19.10,20.90,8.40,6.10,34.20,44.70,51.90,22.20,21.80,25.50,20.30,3.20,40.10,32.40,13.30,4.70,

In [319]:
var_dict[2016] = var_dict[2016].applymap(lambda x: np.nan if x==-666666666 or x==-888888888 or x==-999999999 else x)

In [316]:
var_dict[2016].isnull().sum()

AIANHH         3142
state             0
county            0
AIHHTLI        3142
AITSCE         3142
ANRC           3142
CBSA              0
CD115          3142
CNECTA         3142
CONCIT         3142
COUNTY            0
COUSUB         3142
CSA               0
DIVISION          0
DP02_0001PE       0
DP02_0002PE       0
DP02_0003PE       0
DP02_0004PE       0
DP02_0005PE       0
DP02_0006PE       0
DP02_0007PE       0
DP02_0008PE       0
DP02_0009PE       0
DP02_0010PE       0
DP02_0011PE       0
DP02_0012PE       0
DP02_0013PE       0
DP02_0014PE       0
DP02_0015PE    3142
DP02_0016PE    3142
               ... 
DP05_0079PE       0
DP05_0080PE       0
DP05_0081PE    3142
DP05_0082PE       0
DP05_0083PE       0
DP05_0084PE       0
GEOCOMP           0
GEOVARIANT        0
GEO_ID         3142
LSAD_NAME      3142
METDIV            0
NAME           3142
NECTA          3142
NECTADIV       3142
PLACE          3142
PUMA5          3142
REGION            0
SDELM          3142
SDSEC          3142


In [317]:
var_dict[2016].loc[0, ((var_dict[2016].isnull().sum()>0)&(var_dict[2016].isnull().sum()!=3142))].index

Index(['DP02_0036PE', 'DP02_0037PE', 'DP02_0044PE', 'DP02_0045PE',
       'DP02_0046PE', 'DP02_0047PE', 'DP02_0048PE', 'DP02_0050PE',
       'DP02_0051PE', 'DP02_0094PE', 'DP02_0095PE', 'DP02_0098PE',
       'DP02_0099PE', 'DP02_0101PE', 'DP02_0102PE', 'DP02_0104PE',
       'DP02_0105PE', 'DP02_0106PE', 'DP02_0107PE', 'DP02_0108PE',
       'DP02_0109PE', 'DP03_0015PE', 'DP03_0017PE', 'DP03_0069PE',
       'DP03_0071PE', 'DP03_0073PE', 'DP03_0110PE', 'DP03_0111PE',
       'DP03_0112PE', 'DP03_0113PE', 'DP03_0120PE', 'DP03_0121PE',
       'DP03_0123PE', 'DP03_0124PE', 'DP03_0125PE', 'DP03_0126PE',
       'DP03_0127PE', 'DP03_0129PE', 'DP03_0130PE', 'DP03_0131PE',
       'DP03_0132PE', 'DP04_0075PE', 'DP04_0094PE', 'DP04_0095PE',
       'DP04_0096PE', 'DP04_0097PE', 'DP04_0098PE', 'DP04_0099PE',
       'DP04_0100PE', 'DP04_0111PE', 'DP04_0112PE', 'DP04_0113PE',
       'DP04_0114PE', 'DP04_0115PE'],
      dtype='object')

In [416]:
df = var_dict[2016].drop(columns=var_dict[2016].loc[0, ((var_dict[2016].isnull().sum()>0)&(var_dict[2016].isnull().sum()!=3142))].index)

In [417]:
df = df.loc[:, df.columns.str.contains(r"DP0")].dropna(axis=1)

In [419]:
df_target = var_dict[2016].loc[:, var_dict[2016].loc[0, ((var_dict[2016].isnull().sum()>0)&(var_dict[2016].isnull().sum()!=3142))].index]

In [420]:
df_target.head()

,DP02_0036PE,DP02_0037PE,DP02_0044PE,DP02_0045PE,DP02_0046PE,DP02_0047PE,DP02_0048PE,DP02_0050PE,DP02_0051PE,DP02_0094PE,DP02_0095PE,DP02_0098PE,DP02_0099PE,DP02_0101PE,DP02_0102PE,DP02_0104PE,DP02_0105PE,DP02_0106PE,DP02_0107PE,DP02_0108PE,DP02_0109PE,DP03_0015PE,DP03_0017PE,DP03_0110PE,DP03_0111PE,DP03_0112PE,DP03_0113PE,DP03_0120PE,DP03_0121PE,DP03_0123PE,DP03_0124PE,DP03_0125PE,DP03_0126PE,DP03_0127PE,DP03_0129PE,DP03_0130PE,DP03_0131PE,DP03_0132PE,DP04_0075PE,DP04_0094PE,DP04_0095PE,DP04_0096PE,DP04_0097PE,DP04_0098PE,DP04_0099PE,DP04_0100PE,DP04_0111PE,DP04_0112PE,DP04_0113PE,DP04_0114PE,DP04_0115PE
0,443.00,43.10,80.00,22.00,8.10,15.40,34.50,69.10,66.80,21.30,78.70,4.80,95.20,29.00,71.00,7.50,34.00,3.60,0.00,54.90,0.00,72.80,62.50,58.30,27.70,31.40,41.70,31.50,27.30,15.20,12.00,40.00,53.10,40.80,36.50,35.90,35.80,35.90,2.40,7.70,57.50,21.20,6.60,3.90,0.30,2.80,55.10,16.00,4.90,5.60,18.50
1,260.00,40.00,34.80,8.10,7.10,1.00,18.70,58.30,77.80,26.50,73.50,9.90,90.10,5.60,94.40,11.60,2.60,0.90,0.00,83.30,1.60,55.10,69.00,47.20,20.90,27.80,52.80,28.70,35.90,18.40,16.80,38.90,51.60,85.50,33.60,32.90,31.60,33.40,2.20,6.10,56.40,28.50,6.90,1.00,0.60,0.50,46.20,10.20,7.50,11.80,24.30
2,"1,219.00",37.00,57.90,19.30,16.20,7.50,15.00,61.60,64.80,28.60,71.40,0.00,100.00,27.20,72.80,6.00,28.50,5.40,2.10,53.30,4.70,74.80,69.60,57.40,24.90,34.70,42.60,21.20,29.20,8.50,11.80,35.30,45.00,59.10,27.60,27.40,33.80,25.00,2.70,3.40,46.10,27.50,13.40,5.30,2.20,2.20,59.20,14.80,8.50,4.90,12.60
3,740.00,41.40,65.40,9.00,7.00,15.20,34.30,58.00,70.90,39.50,60.50,0.00,100.00,12.30,87.70,6.00,25.40,1.20,0.00,63.90,3.50,61.00,63.30,46.80,33.40,16.10,53.20,23.60,20.60,14.50,10.00,38.30,49.00,46.70,27.30,27.10,24.60,27.90,2.90,4.80,51.80,31.30,7.80,2.50,0.50,1.30,48.00,15.20,10.60,5.80,20.40
4,"1,050.00",60.50,56.30,9.90,9.70,6.00,30.80,64.10,73.10,29.60,70.40,0.00,100.00,11.40,88.60,13.50,33.80,3.80,2.90,44.60,1.40,68.30,73.00,65.60,29.00,39.00,34.40,27.70,24.10,7.00,8.40,41.30,49.40,48.80,37.00,36.90,45.00,33.80,3.50,2.40,45.70,33.90,12.20,3.50,1.90,0.40,50.80,16.70,8.20,5.10,19.20


In [348]:
# impute_var_df_list = []
# test_scores = []
# for i, col in enumerate(var_dict[2016].loc[0, ((var_dict[2016].isnull().sum()>0)&(var_dict[2016].isnull().sum()!=3142))].index):

#     rfr_impute = RandomForestRegressor()
#     df_temp = df.loc[df_target.iloc[:, i].notnull()]
#     df_target_temp = df_target.iloc[:, i].dropna()

#     X_train, X_test, y_train, y_test = train_test_split(df_temp, df_target_temp)
#     rfr_impute.fit(X_train, y_train)
#     print(col)
#     print("train score: ", rfr_impute.score(X_train, y_train))
#     print("test score: ", rfr_impute.score(X_test, y_test))
#     test_scores.append(rfr_impute.score(X_test, y_test))

#     impute_var_df_list.append(pd.DataFrame(rfr_impute.predict(df), columns=[col]))
# print("average test score: ", np.mean(test_scores))

/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


DP02_0036PE
train score:  0.9892120235383359
test score:  0.929599832357992


/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


DP02_0037PE
train score:  0.8679578078567677
test score:  0.2788970889532306


/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


...

DP04_0113PE
train score:  0.8330661844818806
test score:  0.0721928003921346


/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


DP04_0114PE
train score:  0.8332315900752556
test score:  -0.08565983799232035


/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


DP04_0115PE
train score:  0.9041246970478312
test score:  0.5551166966749481
average test score:  0.3768545234901668


In [350]:
# impute_var_df_list = []
# test_scores = []
# for i, col in enumerate(var_dict[2016].loc[0, ((var_dict[2016].isnull().sum()>0)&(var_dict[2016].isnull().sum()!=3142))].index):

#     dtr_impute = DecisionTreeRegressor()
#     df_temp = df.loc[df_target.iloc[:, i].notnull()]
#     df_target_temp = df_target.iloc[:, i].dropna()

#     X_train, X_test, y_train, y_test = train_test_split(df_temp, df_target_temp)
#     dtr_impute.fit(X_train, y_train)
#     print(col)
#     print("train score: ", dtr_impute.score(X_train, y_train))
#     print("test score: ", dtr_impute.score(X_test, y_test))
#     test_scores.append(dtr_impute.score(X_test, y_test))

#     impute_var_df_list.append(pd.DataFrame(dtr_impute.predict(df), columns=[col]))
# print("average test score: ", np.mean(test_scores))

DP02_0036PE
train score:  1.0
test score:  0.620860263845814
DP02_0037PE
train score:  1.0
test score:  -0.3540257144132797
DP02_0044PE
train score:  1.0
test score:  0.17033203787838214
DP02_0045PE
train score:  1.0
test score:  -0.8235330991911103
DP02_0046PE
train score:  1.0
test score:  -0.8128000559236255
DP02_0047PE
train score:  1.0
test score:  -1.2490716814797262
DP02_0048PE
train score:  1.0
test score:  -0.6965747676040361
DP02_0050PE
train score:  1.0
test score:  -0.7380968673690884
DP02_0051PE
train score:  1.0
test score:  -0.805238008460309
DP02_0094PE
train score:  1.0
test score:  -0.3963149702045534
DP02_0095PE
train score:  1.0
test score:  -0.46272844499213694
DP02_0098PE
train score:  1.0
test score:  -1.2841930632322947
DP02_0099PE
train score:  1.0
test score:  -0.9897324322531109
DP02_0101PE
train score:  1.0
test score:  -0.6068613179220745
DP02_0102PE
train score:  1.0
test score:  -0.6923012171528158
DP02_0104PE
train score:  1.0
test score:  -0.15238377988

In [352]:
# impute_var_df_list = []
# test_scores = []
# for i, col in enumerate(var_dict[2016].loc[0, ((var_dict[2016].isnull().sum()>0)&(var_dict[2016].isnull().sum()!=3142))].index):

#     svr_impute = SVR()
#     df_temp = df.loc[df_target.iloc[:, i].notnull()]
#     df_target_temp = df_target.iloc[:, i].dropna()

#     X_train, X_test, y_train, y_test = train_test_split(df_temp, df_target_temp)
#     svr_impute.fit(X_train, y_train)
#     print(col)
#     print("train score: ", svr_impute.score(X_train, y_train))
#     print("test score: ", svr_impute.score(X_test, y_test))
#     test_scores.append(svr_impute.score(X_test, y_test))

#     impute_var_df_list.append(pd.DataFrame(svr_impute.predict(df), columns=[col]))
# print("average test score: ", np.mean(test_scores))

/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


DP02_0036PE
train score:  -0.04717958697972736
test score:  -0.1146169887781674


/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


DP02_0037PE
train score:  0.07697499178692202
test score:  -0.00249361585518737


/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


...

DP04_0113PE
train score:  0.4109070524588957
test score:  -0.00226239072849177


/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


DP04_0114PE
train score:  0.42050599380409526
test score:  -0.0014201102720277081


/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


DP04_0115PE
train score:  0.1969249039558305
test score:  -0.03462344758590197
average test score:  -0.03248585261319146


In [421]:
impute_var_df_list = []
test_scores = []
for i, col in enumerate(var_dict[2016].loc[0, ((var_dict[2016].isnull().sum()>0)&(var_dict[2016].isnull().sum()!=3142))].index):

    xgr_impute = XGBRegressor()
    df_temp = df.loc[df_target.iloc[:, i].notnull()]
    df_target_temp = df_target.iloc[:, i].dropna()

    X_train, X_test, y_train, y_test = train_test_split(df_temp, df_target_temp)
    xgr_impute.fit(X_train, y_train)
    print(col)
    print("train score: ", xgr_impute.score(X_train, y_train))
    print("test score: ", xgr_impute.score(X_test, y_test))
    test_scores.append(xgr_impute.score(X_test, y_test))

    impute_var_df_list.append(pd.DataFrame(xgr_impute.predict(df), columns=[col]))
print("average test score: ", np.mean(test_scores))

/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[10:00:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
DP02_0036PE
train score:  0.9985899172637263
test score:  0.8950040261999618


/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[10:00:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
DP02_0037PE
train score:  0.6873424490076532
test score:  0.3167814220934839


/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


...

DP04_0113PE
train score:  0.6272027601737549
test score:  0.08473385775812514


/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[10:05:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
DP04_0114PE
train score:  0.6369474632810372
test score:  0.1123555202039973
[10:05:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


DP04_0115PE
train score:  0.8406738353677731
test score:  0.5407470785931667
average test score:  0.43150315531311706


In [422]:
df_impute = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True), impute_var_df_list)

In [423]:
df_impute.describe()

,DP02_0036PE,DP02_0037PE,DP02_0044PE,DP02_0045PE,DP02_0046PE,DP02_0047PE,DP02_0048PE,DP02_0050PE,DP02_0051PE,DP02_0094PE,DP02_0095PE,DP02_0098PE,DP02_0099PE,DP02_0101PE,DP02_0102PE,DP02_0104PE,DP02_0105PE,DP02_0106PE,DP02_0107PE,DP02_0108PE,DP02_0109PE,DP03_0015PE,DP03_0017PE,DP03_0110PE,DP03_0111PE,DP03_0112PE,DP03_0113PE,DP03_0120PE,DP03_0121PE,DP03_0123PE,DP03_0124PE,DP03_0125PE,DP03_0126PE,DP03_0127PE,DP03_0129PE,DP03_0130PE,DP03_0131PE,DP03_0132PE,DP04_0075PE,DP04_0094PE,DP04_0095PE,DP04_0096PE,DP04_0097PE,DP04_0098PE,DP04_0099PE,DP04_0100PE,DP04_0111PE,DP04_0112PE,DP04_0113PE,DP04_0114PE,DP04_0115PE
count,"3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00"
mean,"1,248.67",36.55,48.66,10.37,10.78,7.93,19.84,60.44,72.71,43.43,56.72,7.98,91.72,14.23,85.81,17.97,24.17,3.95,1.03,47.72,5.27,65.00,70.75,57.91,35.08,25.78,42.12,19.74,22.15,9.34,8.64,33.95,44.25,52.98,22.81,22.37,26.20,21.10,2.88,3.90,37.43,31.67,14.70,6.22,2.85,3.17,46.96,15.11,10.05,6.76,21.12
std,"3,725.28",12.16,13.60,5.06,4.85,4.16,7.28,5.62,8.96,11.73,11.81,5.67,5.88,7.51,7.36,11.70,15.12,5.55,3.61,24.60,5.93,7.17,6.65,12.09,11.49,8.95,12.22,8.54,9.45,5.31,6.36,9.34,11.07,12.77,9.92,9.95,10.95,9.65,1.16,2.63,14.61,6.00,6.12,4.63,3.09,5.47,7.10,1.94,1.54,1.34,5.16
min,-43.01,-0.57,-4.86,-1.51,-2.27,-0.95,0.09,9.62,5.88,12.43,-1.21,-2.72,10.34,-0.85,6.63,-5.68,-4.56,-2.25,-0.45,-2.55,-2.25,21.83,21.48,5.86,0.75,-5.93,0.70,0.46,-0.60,-2.18,-1.44,-2.51,-1.21,0.42,0.87,0.54,-0.65,0.35,0.39,-0.36,2.69,1.61,0.88,0.26,-0.05,-0.30,8.18,0.26,0.43,0.48,1.45
25%,128.30,29.20,40.63,8.71,9.14,6.53,15.51,57.97,70.37,34.71,48.30,4.91,89.59,10.00,83.74,9.41,11.82,1.41,0.39,26.80,2.19,60.74,66.95,49.39,27.06,19.56,32.38,13.41,15.66,5.44,4.76,28.06,37.41,45.75,15.43,14.95,18.30,13.96,2.37,2.01,27.49,28.68,9.88,3.05,1.11,0.97,42.55,14.28,9.27,6.15,17.31
50%,296.02,36.00,48.76,9.87,10.33,7.76,19.24,59.89,73.70,43.55,56.47,7.31,92.42,12.58,87.34,16.38,22.75,2.63,0.59,45.92,3.75,65.68,71.09,57.87,33.96,24.48,42.31,19.02,21.35,8.38,7.35,34.07,44.78,54.09,22.12,21.68,25.53,20.19,2.74,3.47,40.47,32.59,13.23,4.55,1.74,1.51,47.28,15.32,10.06,6.65,20.34
75%,802.34,41.85,56.66,10.95,11.52,8.81,23.31,62.34,76.95,51.61,65.59,10.37,94.90,16.15,90.04,24.76,34.50,4.64,0.82,67.10,5.92,69.66,74.88,67.59,41.59,31.72,50.88,24.64,27.55,12.14,10.66,39.73,51.27,60.62,28.49,28.04,32.69,26.42,3.12,5.21,49.26,35.83,18.76,7.65,3.22,2.86,51.48,16.24,10.92,7.29,24.21
max,"69,142.97",97.55,98.80,93.25,86.64,94.28,91.02,97.97,102.82,101.19,95.75,91.99,102.42,89.44,100.72,95.71,91.50,88.54,95.72,106.21,67.23,95.35,95.78,97.78,93.62,80.93,96.32,70.30,90.79,37.80,92.18,71.64,92.87,95.74,66.00,67.26,91.20,64.10,22.09,36.51,96.58,52.69,35.28,31.26,19.94,55.55,96.13,37.86,22.26,26.53,47.13


In [401]:
for i in range(df_target.shape[0]):
    for j in range(df_target.shape[1]):
        if df_target.iloc[i, j]:
            pass
        else:
            print (i, j, df_impute.iloc[i, j])

0 18 0.3701587915420532
0 20 1.4133073091506958
1 18 0.7443952560424805
2 11 9.553938865661621
3 11 7.765506267547607
3 18 0.635379433631897
4 11 7.926748752593994
5 11 4.166656970977783
5 17 4.342953681945801
5 18 0.5773282647132874
5 20 3.0973005294799805
6 11 4.771521091461182
6 15 3.195215940475464
6 17 3.061688184738159
6 18 1.1876146793365479
6 20 1.8477822542190552
6 30 10.951211929321289
6 43 1.5731524229049683
6 45 0.9919759035110474
7 15 0.9721551537513733
7 18 0.0
7 20 0.10659855604171753
7 43 0.6378676295280457
7 44 0.5935386419296265
7 45 0.5166997313499451
8 18 0.40636152029037476
8 20 1.9676765203475952
10 5 7.1631999015808105
10 11 10.722609519958496
10 17 0.6520010828971863
10 20 4.570133686065674
10 33 50.05937194824219
10 44 0.5414832830429077
11 16 13.390957832336426
11 17 1.757449984550476
11 18 0.9278309345245361
11 45 0.6257994174957275
12 18 0.5485392212867737
13 18 0.8752049207687378
13 20 3.698108196258545
13 45 0.6334158182144165
14 17 0.8806463479995728
15 1

153 45 1.972055435180664
154 17 2.0010881423950195
155 4 4.327592849731445
155 5 4.39334774017334
155 11 7.06756591796875
155 17 2.071934461593628
155 18 1.0594490766525269
155 29 2.716822624206543
155 30 3.224801540374756
156 1 17.65973472595215
156 3 0.8007535934448242
156 4 24.221887588500977
156 11 5.118497848510742
156 17 1.293412208557129
156 20 2.9382426738739014
156 30 1.4634850025177002
157 11 2.372089147567749
157 45 0.9069743156433105
161 11 7.50291633605957
163 11 1.7125344276428223
163 18 0.5000321865081787
163 44 1.7266324758529663
164 5 6.355036735534668
173 11 5.115328311920166
173 17 3.1897213459014893
173 18 0.535378098487854
173 20 2.4560208320617676
174 11 3.2885007858276367
174 16 2.441336154937744
174 17 0.7882468700408936
174 18 0.3388378322124481
174 44 0.6064651608467102
174 45 0.7857908606529236
175 11 5.432147979736328
175 17 0.8787326812744141
177 11 6.12630033493042
178 11 3.1526641845703125
178 16 0.0
178 17 1.1438289880752563
178 18 0.2914641797542572
178

323 18 0.2671124339103699
330 18 0.46270498633384705
332 30 8.097962379455566
335 11 2.84592342376709
335 17 0.6756461262702942
335 18 1.059617280960083
335 33 21.439882278442383
336 1 26.33089256286621
336 3 0.5563058853149414
336 4 0.0
336 5 1.3268733024597168
336 8 6.719797134399414
336 18 0.8043961524963379
336 20 10.7062349319458
336 30 2.1336019039154053
342 11 6.720210075378418
342 33 39.11818313598633
356 3 4.7154717445373535
356 4 7.766202449798584
356 5 1.8715993165969849
356 13 3.2569491863250732
356 17 3.2140395641326904
356 18 0.7484955191612244
356 20 4.756424903869629
356 28 6.330774307250977
356 30 9.166027069091797
356 33 12.269742012023926
363 4 26.46792221069336
363 11 6.463300704956055
363 18 1.9141260385513306
370 18 0.355934202671051
372 11 5.80756139755249
372 18 0.9718196988105774
373 11 0.6007257103919983
373 13 6.050117492675781
373 16 1.9431555271148682
373 18 0.843431293964386
373 20 2.642008066177368
374 11 3.599982261657715
374 17 0.09539687633514404
374 2

481 17 0.06029650568962097
481 18 0.42328110337257385
482 11 6.769412517547607
482 17 1.2135796546936035
482 18 0.30897748470306396
482 20 2.0424575805664062
482 30 7.196173191070557
483 4 11.0316162109375
483 17 2.898869514465332
483 18 0.0
483 45 0.8635625839233398
491 18 0.4755377769470215
498 17 0.3158609867095947
506 3 9.105947494506836
506 17 2.565081834793091
506 18 0.43159282207489014
507 18 0.610703706741333
509 5 6.819922924041748
509 11 5.8227949142456055
509 18 0.37409693002700806
511 11 6.735607624053955
512 11 1.5825098752975464
512 17 0.5350787043571472
512 18 0.0
512 20 1.2078843116760254
513 11 3.5800185203552246
513 17 0.0
513 18 0.0
513 20 0.8274810910224915
513 44 0.6709588766098022
513 45 0.8578386306762695
514 4 10.67416763305664
514 5 7.938085556030273
514 11 7.20894718170166
514 18 0.3951874077320099
514 20 0.0
514 44 0.8885560035705566
515 3 5.71953821182251
515 4 7.6190948486328125
515 5 6.349696636199951
515 11 1.3740477561950684
515 13 4.51629638671875
515 1

646 45 0.6870599985122681
648 18 0.40534424781799316
650 4 9.262084007263184
650 11 8.062162399291992
650 17 2.416588306427002
650 18 0.6918632388114929
651 11 2.8825809955596924
651 17 2.6716458797454834
651 18 0.12139272689819336
651 20 4.555563449859619
651 30 3.1339566707611084
652 18 0.5243015289306641
653 3 6.149576187133789
653 5 6.531958103179932
653 11 3.3701207637786865
653 18 0.06611493229866028
653 20 0.0916614830493927
654 9 26.316478729248047
654 17 0.0
654 18 0.23460659384727478
654 19 18.37005043029785
654 20 2.1736135482788086
654 39 1.7753815650939941
655 3 7.4613423347473145
655 11 5.613948822021484
655 17 0.4747675359249115
655 18 0.4794993996620178
655 30 8.176323890686035
656 11 12.020530700683594
656 17 2.2464346885681152
656 18 0.4754548966884613
656 20 1.6687487363815308
657 11 7.1149187088012695
659 18 0.7256447076797485
660 17 5.03596830368042
660 20 2.6211981773376465
660 45 1.1161186695098877
661 17 3.5015501976013184
661 18 0.3351193070411682
662 18 0.6446

...

2566 15 0.525773286819458
2566 17 0.8594701290130615
2566 18 0.0
2566 20 0.7026227116584778
2566 24 19.380693435668945
2566 45 1.1222429275512695
2567 17 0.0
2567 18 0.09000781178474426
2569 18 0.03265342116355896
2569 20 1.5937241315841675
2569 45 0.8489832282066345
2570 5 2.790135383605957
2570 6 4.398261070251465
2570 11 5.832521438598633
2570 15 2.55993914604187
2570 18 0.3214549422264099
2570 20 0.3646085262298584
2570 23 49.88137435913086
2570 24 32.037086486816406
2570 25 4.74732780456543
2570 33 14.80752182006836
2570 45 1.2279255390167236
2570 49 2.8070731163024902
2573 1 11.22728157043457
2573 4 4.9150848388671875
2573 9 38.58513641357422
2573 13 5.7476372718811035
2573 15 4.312306880950928
2573 16 0.0
2573 17 0.0
2573 18 0.0
2573 20 0.0
2573 28 13.613372802734375
2573 30 6.211304664611816
2573 43 0.7092598676681519
2573 44 0.9027078747749329
2573 45 0.20453789830207825
2574 6 10.971915245056152
2574 11 5.972768783569336
2574 16 0.0
2574 17 0.0
2574 18 0.0
2574 20 1.240337133

2664 25 7.178686618804932
2664 29 3.0879061222076416
2664 30 8.69847297668457
2664 43 2.606882095336914
2664 44 0.9773463606834412
2664 45 0.9252302050590515
2665 11 5.525789737701416
2665 17 0.0
2665 18 0.194671630859375
2666 18 0.2273324728012085
2666 20 2.7150895595550537
2666 30 7.187361240386963
2667 5 9.586177825927734
2667 11 3.70660400390625
2667 17 0.11847317218780518
2667 45 1.4886462688446045
2668 18 0.2536892294883728
2669 11 4.483405113220215
2669 18 0.2692197561264038
2669 20 0.35555046796798706
2670 4 16.33020782470703
2670 17 0.9269218444824219
2670 18 0.11881786584854126
2670 20 1.5236754417419434
2670 44 1.5964245796203613
2670 49 3.263913154602051
2672 11 17.01595115661621
2672 13 9.43132495880127
2672 18 0.11541992425918579
2672 30 9.748281478881836
2673 5 4.454233169555664
2673 31 10.823013305664062
2673 32 13.8688325881958
2673 33 13.901229858398438
2673 39 0.8588780164718628
2674 3 3.1971020698547363
2674 6 7.188446521759033
2674 28 2.51651668548584
2674 29 1.960

2781 35 1.759857416152954
2781 36 4.184550762176514
2781 37 1.4535514116287231
2781 43 2.654144048690796
2781 44 3.23933482170105
2781 45 1.1756644248962402
2782 18 0.32920369505882263
2784 17 0.0
2784 18 0.03265342116355896
2785 11 7.659069061279297
2785 18 0.2302895188331604
2786 4 7.31941556930542
2786 5 5.183172225952148
2786 11 4.7253923416137695
2786 13 8.689900398254395
2786 17 0.0
2786 18 0.2797697186470032
2786 20 1.7839316129684448
2786 30 19.527612686157227
2787 3 8.803174018859863
2787 5 7.516599655151367
2787 17 0.0
2787 18 0.0
2787 20 6.225618362426758
2787 30 10.830291748046875
2788 11 2.8135669231414795
2788 13 7.056054592132568
2788 18 0.2568604350090027
2790 3 8.114770889282227
2790 11 3.5824503898620605
2790 17 1.9127038717269897
2790 18 1.050499677658081
2790 20 2.5525147914886475
2790 44 1.9217525720596313
2791 5 4.291065216064453
2791 11 4.735662460327148
2791 15 0.14377498626708984
2791 16 0.8889296054840088
2791 17 0.0
2791 18 0.0
2791 20 0.8312958478927612
2791

2912 18 0.26451295614242554
2912 45 1.6288251876831055
2913 3 5.433533191680908
2913 11 7.442744731903076
2913 18 0.8670197129249573
2913 30 3.532160758972168
2914 11 5.7700653076171875
2916 11 7.84366512298584
2916 17 0.694261908531189
2916 18 0.7717366218566895
2916 20 6.528039455413818
2917 18 1.1439794301986694
2918 11 1.2513102293014526
2918 17 3.52891206741333
2918 30 4.664487361907959
2919 11 4.960488319396973
2919 18 1.025597095489502
2919 45 0.6496229767799377
2920 18 1.056679129600525
2921 11 5.444571018218994
2921 13 8.384830474853516
2921 17 5.6729326248168945
2921 18 0.32239654660224915
2921 20 1.9648038148880005
2921 28 14.439040184020996
2921 30 5.926425933837891
2921 33 28.004924774169922
2921 44 0.8007906079292297
2921 45 0.8040710687637329
2925 17 0.9153904914855957
2926 3 6.001279830932617
2926 4 2.224548816680908
2926 10 1.2076644897460938
2926 11 1.0269566774368286
2926 13 0.0
2926 16 3.7588939666748047
2926 17 12.60295295715332
2926 18 1.2204899787902832
2926 19 1

3086 13 12.362037658691406
3086 17 1.9918855428695679
3086 18 1.6734317541122437
3086 20 6.44291353225708
3086 30 9.190559387207031
3086 44 2.6929407119750977
3089 17 3.127615451812744
3089 18 0.718428373336792
3090 17 1.6882027387619019
3090 18 1.0230047702789307
3093 11 8.093751907348633
3093 17 2.7919631004333496
3093 18 0.5716738104820251
3093 30 5.78743314743042
3094 18 0.7807649374008179
3095 18 0.8174424767494202
3097 17 1.1610667705535889
3099 18 0.7673901319503784
3101 18 0.9263944625854492
3104 17 4.845636367797852
3104 18 1.3466856479644775
3105 18 0.8026683330535889
3107 18 0.7795785665512085
3108 18 0.4504849910736084
3109 17 1.469979166984558
3109 18 0.46412500739097595
3110 5 6.093451976776123
3110 11 5.340542793273926
3110 18 1.4695137739181519
3112 18 1.0960485935211182
3115 18 0.6426414251327515
3116 17 0.9346778988838196
3118 18 0.7729804515838623
3119 4 10.1378755569458
3119 11 10.82402515411377
3120 4 9.720306396484375
3120 11 5.513766288757324
3121 11 6.5053563117

In [400]:
df_impute = df_impute.applymap(lambda x: 0 if x<0 else x).applymap(lambda x: 100 if x>100 else x)

In [402]:
df_impute.describe()

,DP02_0036PE,DP02_0037PE,DP02_0044PE,DP02_0045PE,DP02_0046PE,DP02_0047PE,DP02_0048PE,DP02_0050PE,DP02_0051PE,DP02_0094PE,DP02_0095PE,DP02_0098PE,DP02_0099PE,DP02_0101PE,DP02_0102PE,DP02_0104PE,DP02_0105PE,DP02_0106PE,DP02_0107PE,DP02_0108PE,DP02_0109PE,DP03_0015PE,DP03_0017PE,DP03_0110PE,DP03_0111PE,DP03_0112PE,DP03_0113PE,DP03_0120PE,DP03_0121PE,DP03_0123PE,DP03_0124PE,DP03_0125PE,DP03_0126PE,DP03_0127PE,DP03_0129PE,DP03_0130PE,DP03_0131PE,DP03_0132PE,DP04_0075PE,DP04_0094PE,DP04_0095PE,DP04_0096PE,DP04_0097PE,DP04_0098PE,DP04_0099PE,DP04_0100PE,DP04_0111PE,DP04_0112PE,DP04_0113PE,DP04_0114PE,DP04_0115PE
count,"3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00"
mean,94.40,36.76,48.50,10.45,11.01,7.90,19.81,60.55,72.17,43.28,56.76,8.56,91.36,14.24,85.71,18.09,24.18,3.93,1.09,47.83,5.20,65.04,70.92,58.30,35.15,25.55,42.02,19.77,22.12,9.35,8.70,33.97,44.36,52.59,22.81,22.41,26.24,21.02,2.81,3.95,37.38,31.71,14.75,6.26,2.84,3.18,46.89,15.08,10.05,6.73,21.15
std,13.88,12.43,13.27,5.41,5.19,3.86,7.52,5.55,9.91,11.80,11.54,5.99,6.38,7.26,7.52,11.57,15.19,5.37,3.54,24.56,5.90,7.17,6.70,11.89,11.25,8.86,12.27,8.55,9.37,5.43,6.12,9.41,11.29,12.38,9.92,10.03,11.01,9.66,1.00,2.61,14.60,6.01,6.14,4.66,3.10,5.41,7.20,1.92,1.55,1.34,5.20
min,0.00,0.28,0.00,0.00,0.00,0.00,0.00,9.15,0.00,4.71,1.21,0.00,7.09,0.00,11.45,0.00,0.00,0.00,0.00,0.00,0.00,3.81,33.72,5.06,0.67,0.00,3.04,0.58,0.09,0.00,0.00,0.00,0.14,3.24,0.36,0.10,0.00,0.32,0.70,0.14,2.36,1.84,1.04,0.13,0.00,0.00,7.37,0.63,0.08,0.15,1.81
25%,100.00,28.97,40.74,8.57,9.33,6.47,15.57,58.14,70.10,34.52,48.18,5.42,89.47,10.29,83.58,9.63,11.92,1.38,0.42,26.86,2.32,60.95,67.21,49.52,27.07,19.39,32.19,13.47,15.65,5.43,4.92,28.02,37.57,45.81,15.52,15.05,18.58,14.00,2.36,2.00,27.39,28.85,9.85,3.14,1.10,0.96,42.19,14.23,9.30,6.12,17.21
50%,100.00,36.19,48.85,9.81,10.39,7.74,18.99,59.82,73.21,43.73,56.58,7.74,92.15,12.67,87.27,16.28,22.86,2.66,0.61,45.45,3.76,65.72,71.26,58.04,34.19,23.93,42.34,18.98,21.37,8.35,7.37,34.18,44.79,53.52,21.96,21.71,25.64,20.21,2.73,3.50,40.32,32.72,13.35,4.57,1.71,1.51,47.20,15.30,10.07,6.66,20.37
75%,100.00,42.26,56.44,11.07,11.62,8.77,23.18,62.29,76.37,51.71,65.60,10.62,94.56,16.08,89.93,24.82,34.47,4.75,0.92,67.97,5.57,69.79,75.16,67.72,41.73,31.31,50.45,24.66,27.59,12.12,10.80,40.02,51.61,59.87,28.44,28.03,32.40,26.46,3.07,5.28,49.21,35.82,18.87,7.73,3.14,2.92,51.56,16.23,10.88,7.34,24.31
max,100.00,94.07,97.52,96.62,88.21,90.59,92.27,100.00,99.48,97.76,91.71,88.35,100.00,89.42,100.00,92.54,90.99,95.61,97.68,100.00,67.78,96.05,96.86,95.97,97.27,82.53,98.14,70.97,87.86,56.13,87.29,68.00,92.57,97.72,66.44,66.85,92.18,63.42,22.25,21.20,98.28,52.05,42.72,29.99,20.19,54.32,92.24,30.11,28.47,26.59,45.75


In [367]:
df_target.describe()

,DP02_0036PE,DP02_0037PE,DP02_0044PE,DP02_0045PE,DP02_0046PE,DP02_0047PE,DP02_0048PE,DP02_0050PE,DP02_0051PE,DP02_0094PE,DP02_0095PE,DP02_0098PE,DP02_0099PE,DP02_0101PE,DP02_0102PE,DP02_0104PE,DP02_0105PE,DP02_0106PE,DP02_0107PE,DP02_0108PE,DP02_0109PE,DP03_0015PE,DP03_0017PE,DP03_0110PE,DP03_0111PE,DP03_0112PE,DP03_0113PE,DP03_0120PE,DP03_0121PE,DP03_0123PE,DP03_0124PE,DP03_0125PE,DP03_0126PE,DP03_0127PE,DP03_0129PE,DP03_0130PE,DP03_0131PE,DP03_0132PE,DP04_0075PE,DP04_0094PE,DP04_0095PE,DP04_0096PE,DP04_0097PE,DP04_0098PE,DP04_0099PE,DP04_0100PE,DP04_0111PE,DP04_0112PE,DP04_0113PE,DP04_0114PE,DP04_0115PE
count,"3,117.00","3,107.00","3,124.00","3,124.00","3,124.00","3,124.00","3,124.00","3,091.00","3,091.00","3,129.00","3,129.00","3,067.00","3,067.00","3,129.00","3,129.00","3,129.00","3,129.00","3,129.00","3,129.00","3,129.00","3,129.00","3,140.00","3,141.00","3,132.00","3,132.00","3,132.00","3,132.00","3,141.00","3,135.00","3,140.00","3,131.00","3,141.00","3,140.00","3,059.00","3,141.00","3,141.00","3,140.00","3,141.00","3,138.00","3,141.00","3,141.00","3,141.00","3,141.00","3,141.00","3,141.00","3,141.00","3,141.00","3,141.00","3,141.00","3,141.00","3,141.00"
mean,"1,242.09",36.75,48.88,10.36,10.84,7.86,19.82,60.42,73.13,43.21,56.79,8.26,91.74,14.16,85.84,17.87,24.20,3.89,1.10,47.75,5.19,65.02,70.80,57.90,34.96,25.73,42.10,19.76,22.17,9.36,8.62,33.93,44.30,52.92,22.81,22.40,26.20,21.06,2.85,3.94,37.36,31.72,14.71,6.24,2.85,3.17,46.95,15.06,10.06,6.75,21.19
std,"4,113.39",18.26,16.90,9.85,9.85,7.89,12.57,10.73,16.28,18.70,18.70,12.33,12.33,12.76,12.76,15.77,18.76,7.58,4.64,27.74,8.77,10.63,8.74,16.53,15.32,13.03,16.53,8.91,13.04,5.97,10.01,11.56,14.67,24.44,10.27,10.33,12.64,10.19,1.79,3.30,15.45,7.59,6.88,5.06,3.37,5.63,8.90,3.43,2.86,2.47,6.69
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,16.30,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,123.00,25.10,38.10,4.70,5.30,3.20,12.00,55.00,65.60,30.30,45.50,0.00,88.05,5.80,81.20,6.50,9.50,0.00,0.00,24.90,0.20,58.88,65.90,47.50,24.90,17.00,30.70,13.40,13.20,5.10,2.00,26.70,35.20,37.70,15.40,14.90,17.28,13.70,2.00,1.70,26.70,27.50,9.60,2.80,0.80,0.60,41.10,13.40,8.50,5.40,16.70
50%,298.00,35.90,47.70,8.50,9.00,6.40,18.20,60.30,73.90,42.10,57.90,4.50,95.50,11.50,88.50,14.30,21.10,1.40,0.00,46.60,2.30,65.80,71.50,58.90,34.20,24.65,41.10,18.80,20.50,8.30,5.90,33.80,44.30,52.30,21.90,21.50,25.20,20.00,2.50,3.10,39.70,32.60,13.50,4.80,1.80,1.60,47.20,15.40,10.20,6.70,20.30
75%,816.00,47.10,59.10,13.30,13.60,10.30,25.42,65.70,82.90,54.50,69.70,11.95,100.00,18.80,94.20,25.00,35.10,4.70,0.60,70.30,5.80,71.90,76.50,69.30,43.40,33.90,52.50,24.80,29.00,12.10,11.60,40.90,52.90,68.00,28.60,28.20,33.20,26.70,3.30,5.30,48.90,36.90,19.20,8.10,3.50,3.30,52.60,17.00,11.70,8.10,24.80
max,"121,611.00",100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,83.30,100.00,100.00,100.00,100.00,100.00,100.00,75.10,100.00,61.60,100.00,87.00,100.00,100.00,70.90,70.90,100.00,67.20,25.90,40.50,100.00,65.20,50.10,33.70,21.70,57.70,100.00,43.30,31.80,30.00,100.00


In [403]:
df_impute.shape

(3142, 51)

In [404]:
df_target.shape

(3142, 51)

In [434]:
for i in range(df_target.shape[0]):
    for j in range(df_target.shape[1]):
        if df_target.iloc[i, j] != df_target.iloc[i, j]:
            df_target.iloc[i, j] = df_impute.iloc[i, j]

In [436]:
df_target.describe()

,DP02_0036PE,DP02_0037PE,DP02_0044PE,DP02_0045PE,DP02_0046PE,DP02_0047PE,DP02_0048PE,DP02_0050PE,DP02_0051PE,DP02_0094PE,DP02_0095PE,DP02_0098PE,DP02_0099PE,DP02_0101PE,DP02_0102PE,DP02_0104PE,DP02_0105PE,DP02_0106PE,DP02_0107PE,DP02_0108PE,DP02_0109PE,DP03_0015PE,DP03_0017PE,DP03_0110PE,DP03_0111PE,DP03_0112PE,DP03_0113PE,DP03_0120PE,DP03_0121PE,DP03_0123PE,DP03_0124PE,DP03_0125PE,DP03_0126PE,DP03_0127PE,DP03_0129PE,DP03_0130PE,DP03_0131PE,DP03_0132PE,DP04_0075PE,DP04_0094PE,DP04_0095PE,DP04_0096PE,DP04_0097PE,DP04_0098PE,DP04_0099PE,DP04_0100PE,DP04_0111PE,DP04_0112PE,DP04_0113PE,DP04_0114PE,DP04_0115PE
count,"3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00","3,142.00"
mean,"1,266.61",36.73,48.66,10.41,10.88,7.89,19.84,60.64,72.64,43.27,56.85,8.27,91.71,14.20,85.79,17.97,24.22,3.93,1.10,47.63,5.21,65.02,70.80,57.91,35.02,25.73,42.09,19.76,22.19,9.35,8.63,33.92,44.29,52.92,22.80,22.40,26.19,21.05,2.85,3.94,37.36,31.71,14.71,6.24,2.85,3.17,46.96,15.06,10.06,6.75,21.18
std,"4,158.10",18.21,17.11,9.97,9.87,7.90,12.57,10.82,16.69,18.69,18.70,12.26,12.32,12.79,12.78,15.83,18.76,7.65,4.63,27.75,8.75,10.63,8.74,16.52,15.36,13.03,16.53,8.91,13.03,5.97,10.00,11.57,14.68,24.22,10.27,10.33,12.64,10.19,1.79,3.30,15.45,7.61,6.88,5.06,3.37,5.64,8.91,3.43,2.86,2.47,6.69
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-1.45,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,16.30,0.00,0.00,0.00,0.00,0.00,0.00,-2.18,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,123.00,25.10,37.90,4.73,5.40,3.20,12.00,55.00,64.80,30.40,45.60,0.00,88.10,5.80,81.10,6.50,9.50,0.00,0.00,24.70,0.20,58.90,65.90,47.50,25.00,17.00,30.70,13.40,13.20,5.10,2.00,26.70,35.20,37.80,15.40,14.90,17.20,13.70,2.00,1.70,26.70,27.50,9.60,2.80,0.80,0.60,41.10,13.40,8.50,5.40,16.70
50%,299.00,35.90,47.65,8.50,9.10,6.40,18.30,60.50,73.70,42.10,57.90,4.50,95.40,11.60,88.40,14.40,21.20,1.40,0.00,46.40,2.40,65.80,71.50,58.95,34.30,24.65,41.00,18.80,20.50,8.25,5.90,33.80,44.30,52.35,21.90,21.50,25.20,20.00,2.50,3.10,39.70,32.60,13.50,4.80,1.80,1.60,47.20,15.40,10.20,6.70,20.30
75%,821.00,47.00,59.00,13.30,13.70,10.30,25.50,66.00,82.70,54.50,69.80,11.90,100.00,18.90,94.20,25.30,35.00,4.80,0.60,70.20,5.80,71.90,76.50,69.20,43.48,33.88,52.50,24.80,29.00,12.10,11.60,40.90,52.90,67.80,28.60,28.20,33.20,26.67,3.30,5.30,48.90,36.90,19.17,8.10,3.50,3.30,52.60,17.00,11.70,8.10,24.80
max,"121,611.00",100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,101.16,100.00,100.00,100.00,100.00,100.00,100.00,100.00,83.30,100.00,100.00,100.00,100.00,100.00,100.00,75.10,100.00,61.60,100.00,87.00,100.00,100.00,70.90,70.90,100.00,67.20,25.90,40.50,100.00,65.20,50.10,33.70,21.70,57.70,100.00,43.30,31.80,30.00,100.00


In [463]:
for col in df_target.columns:
    var_dict[2016].loc[:, col] = df_target.loc[:, col]

In [ ]:
missing_row = []
for col in var_dict[2010].:
    if col not in var_dict[2010].columns:
        missing_col.append(col)
missing_col

In [455]:
missing_col = []
for col in var_dict[2016].columns:
    if col not in var_dict[2010].columns:
        missing_col.append(col)
missing_col

['CD115',
 'DP02_0150PE',
 'DP02_0151PE',
 'DP02_0152PE',
 'DP04_0142PE',
 'DP04_0143PE',
 'DP05_0082PE',
 'DP05_0083PE',
 'DP05_0084PE',
 'ZCTA5']

In [461]:
for year in range(2010, 2018):
    var_dict[year] = var_dict[year].applymap(
        lambda x: np.nan if x==-666666666 or x==-888888888 or x==-999999999 else x
    )

In [ ]:
for year in range(2010, 2018):
    df = var_dict[year].drop(
        columns=var_dict[year].loc[
            0, 
            ((var_dict[year].isnull().sum()>0)&(var_dict[year].isnull().sum()!=var_dict[year].shape[0]))
        ].index
    )
    df = df.loc[:, df.columns.str.contains(r"DP0")].dropna(axis=1)
    df_target = var_dict[year].loc[
        :, var_dict[year].loc[
            0,
            ((var_dict[year].isnull().sum()>0)&(var_dict[year].isnull().sum()!=var_dict[year].shape[0]))
        ].index
    ]
    impute_var_df_list = []
    test_scores = []
    for i, col in enumerate(var_dict[year].loc[0, 
                                               ((var_dict[year].isnull().sum()>0)&
                                                (var_dict[year].isnull().sum()!=var_dict[year].shape[0]))
                                              ].index):

        xgr_impute = XGBRegressor()
        df_temp = df.loc[df_target.iloc[:, i].notnull()]
        df_target_temp = df_target.iloc[:, i].dropna()

        X_train, X_test, y_train, y_test = train_test_split(df_temp, df_target_temp)
        xgr_impute.fit(X_train, y_train)
        print(col)
        test_scores.append(xgr_impute.score(X_test, y_test))

        impute_var_df_list.append(pd.DataFrame(xgr_impute.predict(df), columns=[col]))
    print(year, "average test score: ", np.mean(test_scores))
    df_impute = reduce(lambda left, right: pd.merge(
        left, right, left_index=True, right_index=True
    ), impute_var_df_list
                      )
    for i in range(df_target.shape[0]):
        for j in range(df_target.shape[1]):
            if df_target.iloc[i, j] != df_target.iloc[i, j]:
                df_target.iloc[i, j] = df_impute.iloc[i, j]
    for col in df_target.columns:
        var_dict[year].loc[:, col] = df_target.loc[:, col]

/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[11:27:59] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
DP02_0037PE
[11:28:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


DP02_0044PE
[11:28:07] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


...

/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


DP02_0094PE
[11:34:00] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


DP02_0095PE
[11:34:04] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [477]:
for year in range(2010, 2018):
    print(var_dict[year].shape[0])

3143
3143
3143
3143
3142
3142
3142
3142
